In [ ]:
# This requires two dependencies: langchain and langchain_aws
# You also need datasets to use the wikiart-subject dataset
# pip install langchain langchain_aws datasets

# Available models:
# amazon.titan-embed-text-v1
# amazon.titan-embed-image-v1
# anthropic.claude-3-5-sonnet-20240620-v1:0
# cohere.embed-multilingual-v3
# meta.llama3-70b-instruct-v1:0

import boto3
from langchain_aws.chat_models.bedrock import ChatBedrock
from langchain_aws.embeddings.bedrock import BedrockEmbeddings
from datasets import load_dataset
import os

# https://huggingface.co/datasets/jlbaker361/wikiart-subjects
ds = load_dataset("jlbaker361/wikiart-subjects")

session = boto3.Session(
  aws_access_key_id='[AWS_ACCESS_KEY_ID]',
  aws_secret_access_key='[AWS_SECRET_ACCESS_KEY]',
  region_name='us-east-1'
)

client = session.client('bedrock-runtime')